#  TP 6.5 Optimisation basée sur des tuners

## Optimisation basée sur Keras tuner .
Le but est de ne pas teste toutes les solutions possibles (grid solution) (GridSearch).
- for param1 :
    -  for parm2:
        - for parm3
            - ...

Le tuner propose des méthodes qui permettent d'accélérer la recherche de la meilleure solution.
Il propose 4 tuners
- RandomSearch Tuner
- GridSearch Tuner
- BayesianOptimization Tuner
- Hyperband Tuner
- Sklearn Tuner

Il faut aller sur le site de keras (https://keras.io/api/keras_tuner/tuners/), pour comprendre ce que fait chacun de ces tuners (vous pourrez aussi le trouver sur tensorflow (https://www.tensorflow.org/tutorials/keras/keras_tuner).

il faut installer, keras-tuner.


In [8]:
!pip install -q -U keras-tuner

In [9]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import keras_tuner as kt  ## le keras tuner
import pandas as pd
from sklearn.model_selection import train_test_split

### Préparation des données

In [10]:
# Lecture du dataset n permet de limiter le nombre de lignes à lire
# Pour faciliter les tests

def load_data(n):
    data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/train.csv')
    return data[0:n]

def select_variables(data):
    data.dropna(axis=0, inplace=True)
    y = data['smoking'] # récupérer la colonne survived et la mettre dans y
    # récuperer le reste des données dans X utiliser la fonction titanic.drop ???, ??? )
    X = data.drop('smoking', axis=1)
    return X,y

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#### Split des données

In [12]:
def split_data(X,y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    return X_train, X_test, y_train, y_test

In [13]:
X_train[0]

NameError: ignored

In [ ]:
## On peut utiliser une simple normalisation (x-mu)/ecart type)
def normaliser(X_train, X_test):
    mean = X_train.mean()
    std  = X_train.std()
    X_train = (X_train - mean) / std
    X_test  = (X_test  - mean) / std

    return X_train, X_test

In [14]:
# Préparation ds données
# le -1 du load_data(-1) veut dire on prend toutes les lignes
data=load_data(-1)
# sélectionner les variables
X,y = select_variables(data)
X_train, X_test, y_train, y_test = split_data(X,y)
X_train, X_test = normaliser(X_train, X_test)
print("X_train.shape", X_train.shape, "X_test.shape", X_test.shape)

NameError: ignored

#### Définition du modèle.
Je vous propose deux options, j'ai une préférence pour la deuxième option car on peut modifier le nombre de couches

#### Option 1 - les hyperparamètres à l'extérieur du modèle

In [ ]:
def create_model(neurons, lr, activations, hp_optimizers,optimizers_dict ):
    m = X_train.shape[1]
    model = tf.keras.Sequential ()
    model.add(tf.keras.layers.Input(m,name="InputLayer"))
    model.add(tf.keras.layers.Dense(neurons, activation=activations))
    model.add(tf.keras.layers.Dense(neurons, activation=activations))
    model.add(tf.keras.layers.Dense(units=1, activation = 'sigmoid',name='output'))

    model.compile(optimizer=optimizers_dict[hp_optimizers],
                  loss="BinaryCrossentropy",
                  metrics=["accuracy"])
    return model

## Définir les différents paramètres à tester
def build_model_opt1(hp):
    neurons = hp.Int("units", min_value=16, max_value=300, step=16)
    lr = hp.Float("learning_rate", min_value=1e-4, max_value=1e-2)
    #p_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    activations=hp.Choice('activation',values=['tanh' ], default='tanh')
    learning_rate = hp.Float("learning_rate", min_value=1e-4, max_value=1e-1)

    optimizers_dict = {
        "Adam":    tf.keras.optimizers.legacy.Adamax(learning_rate=learning_rate),
        "Adamax":  tf.keras.optimizers.legacy.Adamax(learning_rate=learning_rate),
        "SGD":     tf.keras.optimizers.legacy.SGD(learning_rate=learning_rate),
        "Adagrad": tf.keras.optimizers.legacy.Adagrad(learning_rate=learning_rate)
        }

    hp_optimizers = hp.Choice('optimizer', values=["Adam","Adamax", "SGD", "Adagrad"])

    model = create_model(
        neurons=neurons, lr=lr, activations=activations, hp_optimizers=hp_optimizers, optimizers_dict=optimizers_dict
    )
    return model


### Option 2: les hyperparamètres sont définis dans le modèle

In [ ]:
def build_model_opt2(hp):
    model = tf.keras.Sequential()
    # Tune the number of layers.
    m = X_train.shape[1]

    model = tf.keras.Sequential ()
    # couche d'entrée
    model.add(tf.keras.layers.Input(m,name="InputLayer"))
    # les;couches cachées
    for i in range(hp.Int("num_layers", 1, 5)):
        model.add(
            tf.keras.layers.Dense(
                # Tune number of units separately.
                units=hp.Int(f"units_{i}", min_value=16, max_value=200, step=16),
                activation=hp.Choice("activation", ["relu", "tanh"]),
                )
        )
        # la couche de sortie
    model.add(tf.keras.layers.Dense(1, activation="sigmoid"))

    # Liste hyperparameètres à optimiser
    learning_rate = hp.Float("learning_rate", min_value=1e-4, max_value=1e-1)
    optimizers_dict = {
        "Adam":    tf.keras.optimizers.legacy.Adamax(learning_rate=learning_rate),
        "Adamax":  tf.keras.optimizers.legacy.Adamax(learning_rate=learning_rate),
        "SGD":     tf.keras.optimizers.legacy.SGD(learning_rate=learning_rate),
        "Adagrad": tf.keras.optimizers.legacy.Adagrad(learning_rate=learning_rate)
        }

    hp_optimizers = hp.Choice('optimizer', values=["Adam","Adamax", "SGD", "Adagrad"])


    model.compile(
        optimizer=optimizers_dict[hp_optimizers],
        loss="BinaryCrossentropy",
        metrics=["accuracy"]
    )
    return model


build_model_opt2(kt.HyperParameters())


#### Choix du tuner ainsi que ses paramètres.
Conseil : visiter le site pour visualiser les différents paramètres du tuner (https://keras.io/api/keras_tuner/tuners/base_tuner/#tuner-class)

In [ ]:
#tuner : Gridsearch, RandomSearch, BayesianOptimization, Hyperband
tuner = kt.GridSearch(
    build_model_opt2,
    objective='val_accuracy',
    max_trials=16,
    overwrite=True,
    directory="my_dir",
    project_name="tuning_BN",)

In [ ]:

tuner = kt.Hyperband(
    build_model_opt2,
    objective='val_accuracy',
    max_epochs=16,
    factor=3,
    directory='my_dir',
    project_name='tuning_hyperband')


#### Visualiser les différents paramètres à tester

In [ ]:
tuner.search_space_summary()

#### Lancer la méthode search du tuner avec ses paramètres pour rechercher les best paramètres
Avant de lancer la méthode search on peut aussi lui demander de stopper la recherche si les résultats ne s'améliorent pas, ceci grace à (f.keras.callbacks.EarlyStopping) (https://keras.io/api/callbacks/early_stopping/)

In [ ]:
## cette méthode est utile elle permet de stopper la recherche de solutions
## quand l'erreur (ou la précision, ou ..), variable monitor= la loss, ne s'améliore pas
## au bout de patience=5 epochs

early_stoping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=False)

In [ ]:
tuner.search(X_train, y_train, epochs=32, validation_data=(X_test, y_test), batch_size=32, callbacks=[early_stoping])


In [ ]:
tuner.results_summary()

#### Récupérer (get) les meilleurs paramètres, le meilleur modèle, ....
(https://keras.io/api/keras_tuner/tuners/#the-base-tuner-class)

In [ ]:
# le meilleur modèle est stocké en position [0] du get_best_model
best_model = tuner.get_best_models(num_models=1)[0]

# Les best paramètres
best_hps=tuner.get_best_hyperparameters()[0]

print("best #layers : ",best_hps.get('num_layers'))
print("best learning_rate : ",best_hps.get('learning_rate'))
print("best activation : ",best_hps.get('activation'))
print("best optimizer : ",best_hps.get('optimizer'))
best_model.summary()

#### Comment utiliser le meilleur modèle
- Le meilleur modèle "best_model = tuner.get_best_models()" vient avec le mdèle de neurones déjà entrainé, les paramètres du modèle( W et les b) sont déjà appris. C'est ce que l'on nomme un "checkpoint". Ce modèle est à utiliser directement dans la phase d'évaluation(prédiction)
- Sinon, le best_hps=tuner.get_best_hyperparameters()[0], lui récupère les meilleurs paramètres. Vous pourrez repartir de ces paramètres pour entrainer le modèle. (solution préconisée)


In [ ]:
## Sélectionner les meileurs hyperparamètres du modèle
best_model = tuner.hypermodel.build(best_hps)

# Réentrainer le modèle avec ls nouveaux hyperparamètres
history = best_model.fit(X_train, y_train, epochs=50, validation_data = (X_test, y_test),
                         batch_size=32,
                         verbose=False
                         callbacks=[early_stoping])



In [ ]:
# AH quelle est la meilleure epoch ??
# Réupérer la best epoch
val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))


In [ ]:
history.history

In [ ]:
## On peut aussi sélectionner les meileurs hyperparamètres
best_model = tuner.hypermodel.build(best_hps)

# on réentraine le modèle avec ls nouveaux hyperparamètres
best_model.fit(X_train, y_train, epochs=best_epoch, validation_data = (X_test, y_test),
                         batch_size=32)

In [ ]:
# Utilisation du best model
score = best_model.evaluate(X_test, y_test)

In [ ]:
print("la loss",score[0], "l'accracy", score[1])

In [ ]:
import pandas as pd
pd.DataFrame(history.history).plot(figsize=(15,8))
plt.grid(True)
plt.show()

In [ ]:
y_prob = best_model.predict(X_test)
y_classes = y_prob.argmax(axis=-1)


In [ ]:
confusion_matrix = tf.math.confusion_matrix(y_test, y_classes)

In [ ]:
import seaborn as sb
class_names=[0,1]
# ax = plt.figure(figsize=(8, 6))
fig = sb.heatmap(confusion_matrix,  cmap='Greens')

# labels, title and ticks
fig.set_xlabel('Predicted labels')
fig.set_ylabel('True labels')
fig.set_title('Confusion Matrix')
fig.xaxis.set_ticklabels(class_names)
fig.yaxis.set_ticklabels(class_names)
fig.figure.set_size_inches(5, 5)


plt.show()